## Experiment 4 (section 6.4)
Finding out which keras apps model suits our dataset the best

In [1]:
import os
import random
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
import numpy as np
import matplotlib.image  as mpimg
import matplotlib.pyplot as plt
from datetime import datetime
now = str(datetime.now())
%load_ext tensorboard
import inspect
import zipfile

In [ ]:
os.mkdir('raw')

In [ ]:
zippath='../../datasets/dataset.zip'
with zipfile.ZipFile(zippath, 'r') as zip_ref:
    zip_ref.extractall('raw')

In [4]:
# List all available models
model_dictionary = {m[0]:m[1] for m in inspect.getmembers(tf.keras.applications, inspect.isfunction)}

In [5]:
model_dictionary

{'DenseNet121': <function keras.applications.densenet.DenseNet121(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)>,
 'DenseNet169': <function keras.applications.densenet.DenseNet169(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)>,
 'DenseNet201': <function keras.applications.densenet.DenseNet201(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)>,
 'EfficientNetB0': <function keras.applications.efficientnet.EfficientNetB0(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000, classifier_activation='softmax', **kwargs)>,
 'EfficientNetB1': <function keras.applications.efficientnet.EfficientNetB1(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000, classifier_activation='softmax', **kwargs)>,
 'EfficientNetB2': <function keras.appl

In [7]:
!rm -rf tmp

In [8]:
import os
from shutil import copyfile
from pathlib import Path
import re
import pandas as pd
from tqdm import tqdm

import random
def split_data(SOURCE, TRAINING, VALID, TESTING, splitsizetrain, splitsizevalid, splitsizetest):
    files = []

    os.mkdir(TRAINING)
    os.mkdir(VALID)
    os.mkdir(TESTING)


    for filename in os.listdir(SOURCE):
        file = SOURCE + filename
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print(filename + " is zero length, so ignoring.")

    training_length = int(len(files) * splitsizetrain)
    valid_length = int(len(files) * 0.8)
    #testing_length = int(len(files) - valid_length)
    #print(training_length, valid_length, testing_length)
    shuffled_set = random.sample(files, len(files))

    training_set = shuffled_set[:training_length]

    testing_set = shuffled_set[training_length:valid_length]
    #print(training_length,valid_length)
    valid_set = shuffled_set[valid_length:]


    for filename in training_set:
        this_file = SOURCE + filename
        destination = TRAINING + filename
        copyfile(this_file, destination)

    for filename in valid_set:
        this_file = SOURCE + filename
        destination = VALID + filename
        copyfile(this_file, destination)

    for filename in testing_set:
        this_file = SOURCE + filename
        destination = TESTING + filename
        copyfile(this_file, destination)    

p = Path('raw')
print(p)
folders = [x for x in p.iterdir() if x.is_dir()]
split_size_train = .6
split_size_valid = .2
split_size_test = .2

try:
  os.mkdir('tmp')
except:
  print('error')

try:  
  os.mkdir('tmp/train')
  os.mkdir('tmp/valid')
  os.mkdir('tmp/test')
except:
  print('error but here')  

for f in tqdm(folders):
    z = Path(f)
    source = './'+str(z)+'/'

    split_data(source, './tmp/train/'+z.parts[1]+'/', './tmp/valid/'+z.parts[1]+'/',
                './tmp/test/'+z.parts[1]+'/',split_size_train, split_size_valid, split_size_test)
    



../input/hribike3


100%|██████████| 24/24 [00:43<00:00,  1.81s/it]


In [9]:
BATCHSIZE = 20
TRAINING_DIR = "./tmp/train/"
train_datagen = ImageDataGenerator(rescale=1.0/255.,  #mobilenet requires -1;1 range
                                   rotation_range=40,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size=BATCHSIZE,
                                                    class_mode='categorical',
                                                    target_size=(224, 224))

VALIDATION_DIR = "./tmp/valid/"
validation_datagen = ImageDataGenerator(rescale=1.0/255.)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                              batch_size=BATCHSIZE,
                                                              class_mode='categorical',
                                                              target_size=(224, 224))

TESTING_DIR = "./tmp/test/"
test_datagen = ImageDataGenerator(rescale=1.0/255.)
test_generator = test_datagen.flow_from_directory(TESTING_DIR,
                                                              batch_size=BATCHSIZE,
                                                              shuffle=False,
                                                              class_mode='categorical',
                                                              target_size=(224, 224))
7247
print(len(train_generator.classes))
print(len(validation_generator.classes))
print(len(test_generator.classes))

Found 4338 images belonging to 24 classes.
Found 1462 images belonging to 24 classes.
Found 1447 images belonging to 24 classes.
4338
1462
1447


In [10]:
labels = list(dict.keys(train_generator.class_indices))
(labels)

['Agaricus_arvensis',
 'Amanita_muscaria',
 'Amanita_pantherina',
 'Amanita_phalloides',
 'Auricularia_auricula-judae',
 'Boletus_reticulatus_edulis',
 'Calocera_viscosa',
 'Cantharellus_cibarius',
 'Chlorociboria_aeruginascens',
 'Cortinarius_semisanguineus',
 'Cortinarius_violaceus',
 'Galerina_marginata',
 'Gyromitra_esculenta',
 'Hygrocybe_cantharellus',
 'Lactarius_deliciosus',
 'Leccinum_scabrum',
 'Lepista_nuda',
 'Macrolepiota_procera',
 'Pleurotus_ostreatus',
 'Rubroboletus_satanas',
 'Russula_cyanoxantha',
 'Russula_lepida',
 'Sarcoscypha_austriaca',
 'Suillus_luteus']

In [11]:
from sklearn.utils.class_weight import compute_class_weight
weights = compute_class_weight('balanced', classes=np.unique(train_generator.classes), y=train_generator.classes)
cw = dict(zip( np.unique(train_generator.classes), weights))


In [12]:
print(cw)

{0: 1.1737012987012987, 1: 0.9877049180327869, 2: 1.0954545454545455, 3: 0.926923076923077, 4: 1.063235294117647, 5: 0.9665775401069518, 6: 0.94140625, 7: 0.9717741935483871, 8: 0.9931318681318682, 9: 1.069526627218935, 10: 0.9563492063492064, 11: 0.9986187845303868, 12: 0.9665775401069518, 13: 0.926923076923077, 14: 1.0508720930232558, 15: 0.9877049180327869, 16: 0.9513157894736842, 17: 0.9986187845303868, 18: 1.2819148936170213, 19: 1.1157407407407407, 20: 0.844626168224299, 21: 1.1088957055214723, 22: 0.8689903846153846, 23: 0.9563492063492064}


In [13]:
model_benchmarks = {'model_name': [], 'num_model_params': [], 'validation_accuracy': []}

In [14]:
model_benchmarks

{'model_name': [], 'num_model_params': [], 'validation_accuracy': []}

In [15]:
from tensorflow.keras import applications
def getGenerators(modelname):
    PF = None
    TS = (224, 224)
    BATCHSIZE = 20
    if 'DenseNet' in modelname:
        PF = applications.densenet.preprocess_input
    elif 'InceptionResNetV2' in modelname:
        PF = applications.inception_resnet_v2.preprocess_input
    elif 'InceptionV3' in modelname: 
        PF = applications.inception_v3.preprocess_input
    elif 'MobileNet' in modelname:
        PF = applications.mobilenet.preprocess_input
    elif 'MobileNetV2' in modelname:
        PF = applications.mobilenet_v2.preprocess_input
    elif 'NASNetLarge' in modelname:
        PF = applications.nasnet.preprocess_input
        TS = (331, 331)
    elif 'NASNetMobile' in modelname:
        PF = applications.nasnet.preprocess_input
    elif 'ResNet101' in modelname:
        PF = applications.resnet50.preprocess_input
    elif 'ResNet101V2' in modelname:
        PF = applications.resnet_v2.preprocess_input
    elif 'ResNet152' in modelname:
        PF = applications.resnet50.preprocess_input
    elif 'ResNet152V2' in modelname:
        PF = applications.resnet_v2.preprocess_input
    elif 'ResNet50' in modelname:
        PF = applications.resnet50.preprocess_input
    elif 'ResNet50V2' in modelname:
        PF = applications.resnet_v2.preprocess_input
    elif 'VGG16' in modelname:
        PF = applications.vgg16.preprocess_input
    elif 'VGG19' in modelname:
        PF = applications.vgg19.preprocess_input
    elif 'Xception' in modelname:
        PF = applications.xception.preprocess_input   
    
    train_datagen = ImageDataGenerator(preprocessing_function=PF, 
                                   rotation_range=40,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    fill_mode='nearest')
    
    train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                        batch_size=BATCHSIZE,
                                                        class_mode='categorical',
                                                        target_size=TS)
    
    validation_datagen = ImageDataGenerator(preprocessing_function=PF)
    
    validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                                  batch_size=BATCHSIZE,
                                                                  class_mode='categorical',
                                                                  target_size=TS)

    return train_generator, validation_generator, TS
        

In [16]:
# Loop over each model available in Keras
from tensorflow.keras import Model

for model_name, model in (model_dictionary.items()):
    print('____________________________________________________________________')
    print(str(model_name))
    traingen, validgen, TS = getGenerators(str(model_name))
    
        
    # load the pre-trained model with global average pooling as the last layer and freeze the model weights
    pre_trained_model = model(include_top=False, weights="imagenet", input_shape=TS+(3,))
    for layer in pre_trained_model.layers:
       layer.trainable = False

    # custom modifications on top of pre-trained model
    x = tf.keras.layers.Flatten()(pre_trained_model.output)
    x = tf.keras.layers.Dense(1024, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)      
    x = tf.keras.layers.Dense(24, activation='softmax')(x)           

    model = Model(pre_trained_model.input, x) 

    model.compile(optimizer = RMSprop(learning_rate=0.00001), 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])
    
    history = model.fit(traingen, epochs=3,class_weight=cw, validation_data=validgen)

   
    model_benchmarks['model_name'].append(model_name)
    model_benchmarks['num_model_params'].append(pre_trained_model.count_params())
    model_benchmarks['validation_accuracy'].append(history.history['val_accuracy'][-1])
    print('____________________________________________________________________')

____________________________________________________________________
DenseNet121
Found 4338 images belonging to 24 classes.
Found 1462 images belonging to 24 classes.


2021-12-02 14:04:53.278388: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-02 14:04:53.524687: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-02 14:04:53.525814: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-02 14:04:53.528234: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

29097984/29084464 [==============================] - 0s 0us/step


2021-12-02 14:05:00.978912: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/3


2021-12-02 14:05:09.774375: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


217/217 [==============================] - 104s 415ms/step - loss: 2.3662 - accuracy: 0.3628 - val_loss: 1.0836 - val_accuracy: 0.6888
Epoch 2/3
217/217 [==============================] - 87s 397ms/step - loss: 1.1771 - accuracy: 0.6478 - val_loss: 0.7223 - val_accuracy: 0.7791
Epoch 3/3
217/217 [==============================] - 89s 408ms/step - loss: 0.9152 - accuracy: 0.7185 - val_loss: 0.6581 - val_accuracy: 0.7948
____________________________________________________________________
____________________________________________________________________
DenseNet169
Found 4338 images belonging to 24 classes.
Found 1462 images belonging to 24 classes.
51888128/51877672 [==============================] - 0s 0us/step
Epoch 1/3
217/217 [==============================] - 105s 439ms/step - loss: 2.0660 - accuracy: 0.4308 - val_loss: 0.8966 - val_accuracy: 0.7291
Epoch 2/3
217/217 [==============================] - 92s 423ms/step - loss: 0.9806 - accuracy: 0.6992 - val_loss: 0.6618 - val_accu

2021-12-02 15:56:13.024225: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.91GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.


Epoch 2/3
217/217 [==============================] - 200s 922ms/step - loss: 1.4506 - accuracy: 0.5726 - val_loss: 1.2660 - val_accuracy: 0.6047
Epoch 3/3
217/217 [==============================] - 200s 922ms/step - loss: 1.2684 - accuracy: 0.6330 - val_loss: 1.1558 - val_accuracy: 0.6402
____________________________________________________________________
____________________________________________________________________
NASNetMobile
Found 4338 images belonging to 24 classes.
Found 1462 images belonging to 24 classes.
20004864/19993432 [==============================] - 1s 0us/step
Epoch 1/3
217/217 [==============================] - 110s 428ms/step - loss: 2.2277 - accuracy: 0.3757 - val_loss: 1.5872 - val_accuracy: 0.4938
Epoch 2/3
217/217 [==============================] - 90s 414ms/step - loss: 1.4629 - accuracy: 0.5609 - val_loss: 1.3061 - val_accuracy: 0.5869
Epoch 3/3
217/217 [==============================] - 95s 439ms/step - loss: 1.1810 - accuracy: 0.6360 - val_loss: 1.029

In [17]:
displaydf = pd.DataFrame(model_benchmarks)
displaydf.sort_values('validation_accuracy', inplace=True)
displaydf

,model_name,num_model_params,validation_accuracy
16,MobileNetV3Small,1529968,0.069083
14,MobileNetV2,2257984,0.774966
13,MobileNet,3228864,0.749658
3,EfficientNetB0,4049571,0.795486
15,MobileNetV3Large,4226432,0.101915
18,NASNetMobile,4269716,0.668263
4,EfficientNetB1,6575239,0.817373
0,DenseNet121,7037504,0.794802
5,EfficientNetB2,7768569,0.796854
6,EfficientNetB3,10783535,0.796854
